In [46]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import re
from tensorflow import keras
import time
import pickle

In [2]:
def sorted_alphanumeric(data):
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [ convert(c) for c in re.split('([0-9]+)', key) ]
    return sorted(data, key=alphanum_key)

In [3]:
def load_img(src):
    img = cv2.imread(src)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if not img.shape == (32, 32):
        img = cv2.resize(img, (32, 32))
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
    img = cv2.bitwise_not(img)
    img = img.flatten()
    img = img.astype('int32')
    return img

In [ ]:
start = time.time()

char_dict = {"char": [], "matrix": []}
char_set = dict()
src = r"dataset/handwritten_letters"
for i in range(500):
    for folder in os.listdir(src):
        char_dict["char"].append(folder)
        if folder not in char_set:
            char_set[folder] = sorted_alphanumeric(os.listdir(src + '/' + folder))
        img = load_img(f"{src}/{folder}/{char_set[folder][i]}")
        char_dict["matrix"].append(img)


stop = time.time()
print(stop-start, 'seconds')

In [5]:
import category_encoders as ce
le =  ce.OneHotEncoder(return_df=False, handle_unknown="ignore")
y = np.array(char_dict['char'])

y = le.fit_transform(y)
X = np.array(char_dict["matrix"])

/Users/marcuseriksson/opt/anaconda3/envs/tf/lib/python3.7/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [6]:
split = int(round(0.9 * len(X)))

train_X = X[:split]/255
train_y = y[:split]
test_X = X[split:]/255
test_y = y[split:]

In [7]:
train_X = train_X.astype('int32')
test_X = test_X.astype('int32')

In [8]:
values = []
for char in train_X:
    for value in char:
        if value not in values:
            values.append(value)
print('x values:', values)
values = []
for char in train_y:
    for value in char:
        if value not in values:
            values.append(value)
print('y values:', values)

x values: [0, 1]
y values: [1, 0]


In [9]:
train_X = tf.constant(train_X)
train_y = tf.constant(train_y)

test_X = tf.constant(test_X)
test_y = tf.constant(test_y)

In [10]:
train_X

<tf.Tensor: id=0, shape=(15750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [11]:
train_y

<tf.Tensor: id=1, shape=(15750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [12]:
test_X

<tf.Tensor: id=2, shape=(1750, 1024), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>

In [13]:
test_y

<tf.Tensor: id=3, shape=(1750, 35), dtype=int64, numpy=
array([[1, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 1]])>

In [14]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(500, activation='relu', input_dim=1024))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(35, activation='softmax'))

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [15]:
model.fit(train_X, train_y, epochs=20)

Train on 15750 samples
Epoch 1/20
15750/15750 [==============================] - 5s 330us/sample - loss: 1.3279 - accuracy: 0.6275
Epoch 2/20
15750/15750 [==============================] - 5s 294us/sample - loss: 0.5911 - accuracy: 0.8187
Epoch 3/20
15750/15750 [==============================] - 5s 335us/sample - loss: 0.3975 - accuracy: 0.8754
Epoch 4/20
15750/15750 [==============================] - 5s 321us/sample - loss: 0.2925 - accuracy: 0.9067
Epoch 5/20
15750/15750 [==============================] - 5s 319us/sample - loss: 0.2277 - accuracy: 0.9244
Epoch 6/20
15750/15750 [==============================] - 5s 338us/sample - loss: 0.1753 - accuracy: 0.9425
Epoch 7/20
15750/15750 [==============================] - 5s 323us/sample - loss: 0.1451 - accuracy: 0.9514
Epoch 8/20
15750/15750 [==============================] - 5s 316us/sample - loss: 0.1179 - accuracy: 0.9594
Epoch 9/20
15750/15750 [==============================] - 6s 355us/sample - loss: 0.1052 - accuracy: 0.9642
Epoch

In [20]:
pred_train= model.predict(train_X)
scores = model.evaluate(train_X, train_y, verbose=0)
print(f'Accuracy on training data: {scores[1]}% \n Error on test data: {1 - scores[1]}')   
print()
pred_test= model.predict(test_X)
scores2 = model.evaluate(test_X, test_y, verbose=0)
print(f'Accuracy on test data: {scores2[1]}% \n Error on test data: {1 - scores2[1]}')    

Accuracy on training data: 0.9846984148025513% 
 Error on test data: 0.01530158519744873

Accuracy on test data: 0.8285714387893677% 
 Error on test data: 0.17142856121063232


In [38]:
def get_wrong_predictions(preds):
    in_test_y = char_dict['char'][split:]
    chars = char_dict['char'][:35]
    print("{Actual letter} - {Predicted letter}")
    for i, prediction in enumerate(preds):
        prediction = list(prediction)
        actual_letter = in_test_y[i]
        predicted_letter = chars[prediction.index(max(prediction))]
        if predicted_letter != actual_letter:
            print(actual_letter, "-", predicted_letter)

In [39]:
get_wrong_predictions(pred_test)

{Actual letter} - {Predicted letter}
G - Q
A - H
B - J
K - X
9 - Q
N - A
G - S
6 - B
S - J
V - Y
9 - Q
G - 9
S - J
4 - Y
E - 8
K - X
W - U
R - N
U - V
9 - Q
S - J
H - V
D - Q
V - W
W - U
G - C
J - 2
9 - Q
I - J
S - J
M - Q
U - V
9 - Q
I - 1
G - 7
F - T
C - U
4 - H
P - J
R - A
9 - Q
I - 1
G - C
2 - G
U - V
I - 1
N - U
Z - T
S - J
F - G
H - B
4 - U
B - G
K - X
2 - Z
R - D
D - Q
X - Y
2 - S
Y - X
T - F
A - G
4 - H
E - 8
7 - T
I - 1
T - 8
S - J
A - H
C - U
D - N
0 - C
I - J
N - H
G - P
A - H
Q - A
P - R
N - W
6 - B
Q - 9
P - Q
9 - Q
8 - R
S - V
A - H
C - U
D - T
Y - V
1 - I
H - U
Q - Y
B - H
K - B
2 - Q
Y - X
P - A
9 - 7
G - S
S - 5
A - H
F - 6
4 - Y
2 - R
Y - X
U - V
I - J
S - J
4 - U
K - Z
2 - D
5 - C
P - Q
9 - Q
G - J
P - D
W - U
9 - Q
N - H
G - Q
4 - U
E - 0
B - D
L - H
2 - D
P - 9
I - L
2 - D
Y - R
D - N
Q - E
2 - G
9 - Q
0 - A
8 - Q
D - U
P - D
9 - Q
A - R
4 - U
0 - C
8 - B
T - Y
F - S
B - R
2 - C
I - 1
G - 6
Z - X
T - H
Q - G
2 - S
W - N
I - 1
Q - D
K - R
R - T
G - C
1 - I
F - 6
P -

In [42]:
def predict_single_img(src):
    img = load_img(src)
    chars = char_dict['char'][:35]
    prediction = model.predict(np.array([img,])/255)
    prediction = list(prediction[0])
    return chars[prediction.index(max(prediction))]

In [43]:
src = 'dataset/handwritten_letters/H/_1_11.jpg'
predict_single_img(src)

'H'